In [1]:
from PIL import Image, ImageOps, ImageFont
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets
from ipywidgets import interactive, HBox, VBox
from mono_char_art_conversion_wxh import quantize_grayscale_wxh
from img_processing import preprocess_img, DITHER_MODES
from braille_art import get_braille_chars
from generate_char_palette import *
from mono_char_art_conversion_mlp import train_classifier

In [2]:
ASCII_SET = get_asciis()
BRAILLE_SET = get_braille_chars()

CHAR_SET = ASCII_SET
W_H_WIN_SHAPE = (2,4)

In [3]:
FONT = ImageFont.truetype("fonts/CascadiaMono.ttf", 12)
IMG_PATH = "imgs/irad_grad.bmp"

In [4]:
orig_img = Image.open(IMG_PATH).convert("L")
#orig_img = ImageOps.invert(orig_img)

print(orig_img.size)

(512, 512)


In [5]:
def i_preprocess_img(scale_factor, contrast, brightness, quantize_colors, eq, dither):
    proc_img = preprocess_img(orig_img, scale_factor, contrast, brightness, eq, quantize_colors, dither)
    plt.imshow(proc_img, cmap='gray', vmin=0, vmax=255, interpolation='none')

    plt.figure(figsize=(9, 3))
    plt.xticks([x for x in range(0, len(proc_img.histogram()), 25)])
    plt.bar([x for x in range(0, len(proc_img.histogram()))], proc_img.histogram())
    plt.grid()
    plt.show()

    return proc_img

interactive_preprocess = interactive(i_preprocess_img,
            scale_factor=(0.01, 1, 0.01),
            contrast=(0, 2, 0.01),
            brightness=(0, 2, 0.01),
            eq=(0, 1, 0.01),
            quantize_colors=(1, 256, 1),
            dither=DITHER_MODES)

controls = VBox(interactive_preprocess.children[:-1])
display(HBox((controls, interactive_preprocess.children[-1])))

In [6]:
proc_img = interactive_preprocess.result

In [7]:


char_to_brightness_map = generate_brightness_map(CHAR_SET, FONT, W_H_WIN_SHAPE, normalize=True)
ascii_arr = quantize_grayscale_wxh(proc_img, char_to_brightness_map, (W_H_WIN_SHAPE[1], W_H_WIN_SHAPE[0]), DITHER_MODES.FS, 0.0)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

      .....:::::::::::::::|:||||||||||||]|]|]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]|]||||||||||||||::::::::::::::::.....     
    ....::::::::::::::|:|||||||||||]|]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]|]|||||||||||:::::::::::::::.....  
  ....:::::::::::::|:||||||||||]|]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]|]||||||||||::::::::::::::.... 
....:::::::::::::|:||||||||]|]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]||||||||||:|::::::::::::...
..:::::::::::::|||||||||]|]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]||||||||||:::::::::::::.
.::::::::::::|||||||||]|]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]||||||||||::::::::::::
:::::::::::||||||||]|]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]|||||||||::::::::::
:::::::::||||||||]|]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]]

In [8]:
noise = 0.3
cls, char_to_brightness_map = train_classifier(CHAR_SET, FONT, W_H_WIN_SHAPE, (8,8,8), 500, noise)

brightness_X = np.array([b.flatten() for b in char_to_brightness_map.values()])
brightness_y = list(char_to_brightness_map.keys())

repetitions = 50

test_X = np.tile(brightness_X, (repetitions,1))
test_y = list(char_to_brightness_map.keys()) * repetitions
test_X += ((np.random.random(test_X.shape)-0.5) * noise)
test_X = np.clip(test_X, a_min=0, a_max=1)

print('classifier score no noise', cls.score(brightness_X, brightness_y))
print('classifier score noise', cls.score(test_X, test_y))

ascii_arr = quantize_grayscale_wxh(proc_img, char_to_brightness_map, (W_H_WIN_SHAPE[1], W_H_WIN_SHAPE[0]), DITHER_MODES.FS, cls=cls)
for i in range(len(ascii_arr)):
    for j in range(len(ascii_arr[i])):
        print(ascii_arr[i][j], sep='', end='')
    print('\n', end='')

classifier score no noise 0.968421052631579
classifier score noise 0.4368421052631579
    :   .... `.:::;;;;::;;;;;;;!|;|;|;/|||]||]|]]|]||])]{j]]]]]]]]|]|]|]]]|]|]]|]|]|\|})\|;||;|;|/;;;;;;;;;:;;,:::: . .. .. .  
     .   `.::;:;;/;;;;;;;;;||;]|;|||])|}]]]|]||]{]}|]]j|])]]}}j)]}j]]]{j]]]]]]]])]]]]]]])])|\||;/|;|;;;|;;!;;!;:;;:::.;.: ..  .
:. .....:..::;:;!;;:;;;;/;);));{]\]]]]]|]|]]]]j]]]j]]]]]j]]]]]]|]]]]]]j]]]]|][]]1|]}]|)||}i]|||]||)|||;;;;;:;:;;;::::::..... . 
 .  .::.;:::;;:;;;;:;;\;;[|]}|]|]]|]]|]|jj|]]{]]]]]])]]])]]]]|]]]]]j]]]]]}]]j]]]j]]]]]j]]j{|]])|||;||||||;;;|;;;;:;;;:.::.: . .
 . ...-.::;:;!;;;l;|;||||];||]])|)]]]]]]]]]j]j]]]]]]j]]]]]]]j]]]jj]]]]j]]j]]j]]]]]j]]]]]]]]])]]]]]|]||;||)|;/;|!;;;;:;:;:;:-:.:
 ..;:.;;::;!;:;!;/|;|]/||]]]]]]]]]])]]]j]]j]]]]j]j]]]j]j]]j]]]]j]]]]]j]]]]]]]]]]]jj]]j]]]]])})]j]||]|]]||)||||;;|;|;:;;:;:;:.::
..:.;::!;!;;;;;;|;||/|]]|]|]|]|]]]]]]]]]]]]]]j]]]]]j]]j]]]]jj]]j]j]]]]]]]]]]j]j]]]]jj]]]]]]]]]]j]]|]]]]))]])|;|;|/;;;;;;::';:::
.:.;:;;;;;;!;|;)\]